In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("./mnist/data/", one_hot=True)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [4]:
learning_rate=0.0001
total_epoch=300
batch_size=100
n_hidden=256
n_input=28*28
n_noise=128

X=tf.placeholder(tf.float32, [None, n_input])
Z=tf.placeholder(tf.float32, [None, n_noise])

G_W1=tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1=tf.Variable(tf.zeros([n_hidden]))
G_W2=tf.Variable(tf.random_normal([n_hidden,n_input],stddev=0.01))
G_b2=tf.Variable(tf.zeros([n_input]))

D_W1=tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1=tf.Variable(tf.zeros([n_hidden]))
D_W2=tf.Variable(tf.random_normal([n_hidden,1],stddev=0.01))
D_b2=tf.Variable(tf.zeros([1]))

In [5]:
def generator(noise_z): #define generator
    hidden=tf.nn.relu(tf.matmul(noise_z, G_W1)+G_b1)
    output=tf.nn.sigmoid(tf.matmul(hidden,G_W2)+G_b2)
    return output

In [6]:
def discriminator(inputs): #define discriminator
    hidden=tf.nn.relu(tf.matmul(inputs, D_W1)+D_b1)
    output=tf.nn.sigmoid(tf.matmul(hidden,D_W2)+D_b2)
    return output

In [7]:
def get_noise(batch_size, n_noise): #input generator
    return np.random.normal(size=(batch_size, n_noise))

In [8]:
G=generator(Z) #fake data 생성 
D_gene=discriminator(G) #fake data를 판별한값
D_real=discriminator(X) #real data를 판별한값

In [9]:
loss_D=tf.reduce_mean(tf.log(D_real)+tf.log(1-D_gene)) #D_real을 1로 D_gene를 0으로 만들기위한 구조
loss_G=tf.reduce_mean(tf.log(D_gene)) #D_gene를 1로 만들기 위한 loss  더 커지게 (음수로 커짐)

In [10]:
D_var_list=[D_W1, D_b1, D_W2, D_b2]
G_var_list=[G_W1, G_b1, G_W2, G_b2]

In [11]:
train_D=tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G=tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)
#각 신경망 입장에서 loss가 커야 더 잘 학습된 상태를 나타냄.
#때문에 Optimizer의 minimize(최소화) 함수에 전달할때 마이너스 부호를 붙힘.
#var_list는 생성기 학습할때는 생성기를 구성하는 파라미터만, 판별기를 학습할때는 판별기를 구성하는 파라미터만 조정하도록함.

In [12]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    total_batch=int(mnist.train.num_examples/ batch_size)
    
    for epoch in range(total_epoch):
        loss_val_D, loss_val_G = 0, 0
        
        for i in range(total_batch):
            batch_xs, batch_ys= mnist.train.next_batch(batch_size)
            noise=get_noise(batch_size, n_noise)
            
            _, loss_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs, Z:noise})
            _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z:noise})
        
        print('Epoch:', '%04d' % epoch, 'D loss: {:.4}'.format(loss_val_D), 'G loss: {:.4}'.format(loss_val_G))
        
        if epoch ==0 or (epoch+1) % 10 == 0:
            sample_size=10
            noise = get_noise(sample_size, n_noise)
            samples=sess.run(G, feed_dict={Z:noise})
            
            fig,ax=plt.subplots(1, sample_size, figsize=(sample_size,1))
            
            for i in range(sample_size):
                ax[i].set_axis_off()
                ax[i].imshow(np.reshape(samples[i],(28,28)))
                
            plt.savefig('{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
            
            plt.close(fig)

Epoch: 0000 D loss: -0.8633 G loss: -1.518
Epoch: 0001 D loss: -0.706 G loss: -1.612
Epoch: 0002 D loss: -0.5782 G loss: -1.638
Epoch: 0003 D loss: -0.3913 G loss: -1.769
Epoch: 0004 D loss: -0.6197 G loss: -1.414
Epoch: 0005 D loss: -0.4195 G loss: -1.634
Epoch: 0006 D loss: -0.6205 G loss: -1.34
Epoch: 0007 D loss: -0.5107 G loss: -1.663
Epoch: 0008 D loss: -0.4732 G loss: -1.94
Epoch: 0009 D loss: -0.379 G loss: -2.167
Epoch: 0010 D loss: -0.2275 G loss: -2.562
Epoch: 0011 D loss: -0.2011 G loss: -2.802
Epoch: 0012 D loss: -0.5506 G loss: -1.983
Epoch: 0013 D loss: -0.3292 G loss: -2.103
Epoch: 0014 D loss: -0.4515 G loss: -2.111
Epoch: 0015 D loss: -0.4451 G loss: -2.01
Epoch: 0016 D loss: -0.4265 G loss: -2.248
Epoch: 0017 D loss: -0.6348 G loss: -1.771
Epoch: 0018 D loss: -0.4357 G loss: -2.11
Epoch: 0019 D loss: -0.4358 G loss: -2.186
Epoch: 0020 D loss: -0.5277 G loss: -1.947
Epoch: 0021 D loss: -0.7216 G loss: -1.619
Epoch: 0022 D loss: -0.5653 G loss: -1.951
Epoch: 0023 D los

Epoch: 0192 D loss: -0.2223 G loss: -3.155
Epoch: 0193 D loss: -0.3971 G loss: -2.864
Epoch: 0194 D loss: -0.5895 G loss: -2.127
Epoch: 0195 D loss: -0.3555 G loss: -2.817
Epoch: 0196 D loss: -0.4983 G loss: -2.529
Epoch: 0197 D loss: -0.4845 G loss: -2.475
Epoch: 0198 D loss: -0.4554 G loss: -2.311
Epoch: 0199 D loss: -0.3856 G loss: -2.829
Epoch: 0200 D loss: -0.5306 G loss: -2.915
Epoch: 0201 D loss: -0.4682 G loss: -2.748
Epoch: 0202 D loss: -0.5002 G loss: -2.806
Epoch: 0203 D loss: -0.4382 G loss: -2.812
Epoch: 0204 D loss: -0.4728 G loss: -3.009
Epoch: 0205 D loss: -0.4251 G loss: -2.65
Epoch: 0206 D loss: -0.3938 G loss: -2.59
Epoch: 0207 D loss: -0.4403 G loss: -2.776
Epoch: 0208 D loss: -0.4044 G loss: -2.836
Epoch: 0209 D loss: -0.4667 G loss: -2.31
Epoch: 0210 D loss: -0.4287 G loss: -2.598
Epoch: 0211 D loss: -0.4767 G loss: -2.791
Epoch: 0212 D loss: -0.521 G loss: -2.542
Epoch: 0213 D loss: -0.468 G loss: -2.848
Epoch: 0214 D loss: -0.3849 G loss: -2.647
Epoch: 0215 D lo